In [5]:
import json
import pandas as pd


df = pd.read_csv('MPEA_dataset.csv')
df_ys = df.dropna(subset=['PROPERTY: YS (MPa)'])
df_ys_FCC = df_ys[df_ys['PROPERTY: BCC/FCC/other'] == 'FCC']

samples = df_ys_FCC.groupby(['FORMULA', 'PROPERTY: Processing method', 'REFERENCE: doi'])
for s in samples:
    if len(s[1]) > 1:
        print(s[0], len(s[1]), [t for t in s[1]['PROPERTY: Test temperature ($^\circ$C)']])
df

('Al0.1 Co1 Cr1 Fe1 Ni1', 'WROUGHT', '10.1016/j.actamat.2018.12.012') 4 [25.0, 25.0, 25.0, 25.0]
('Al0.108 Co0.514 Cr0.514 Cu0.108 Fe0.514 Ni1', 'WROUGHT', '10.1016/j.matchar.2018.06.036') 2 [25.0, 25.0]
('Al0.143 Cr0.343 Fe1 Mn0.8 Ni0.571', 'CAST', '10.1016/j.msea.2019.05.056') 2 [25.0, 25.0]
('Al0.278 Co0.694 Cr0.222 Fe0.417 Ni1 Ti0.167', 'ANNEAL', '10.3390/e21020169') 10 [23.0, 600.0, 700.0, 800.0, 900.0, 23.0, 600.0, 700.0, 800.0, 900.0]
('Al0.286 Cr0.343 Fe1 Mn0.657 Ni0.571', 'WROUGHT', '10.1016/j.msea.2019.05.056') 2 [25.0, 25.0]
('Al0.304 Co1 Cr1 Fe1 Ni1', 'WROUGHT', '10.1016/j.matchar.2018.06.036') 2 [25.0, 25.0]
('Al0.5 Co1 Cr1 Cu1 Fe1 Ni1', 'CAST', '10.1007/s11661-005-0218-9') 7 [1000.0, 25.0, 300.0, 500.0, 700.0, 900.0, 1100.0]
('Al0.5 Co1 Cr1 Cu1 Fe1 Ni1', 'WROUGHT', '10.1016/j.matchar.2018.06.036') 2 [25.0, 700.0]
('Al1 Cu0.889 Fe0.667 Ni0.889 Ti1', 'CAST', '10.1016/j.apsusc.2015.07.207') 2 [25.0, 25.0]
('Co0.25 Cr0.25 Fe1 Mn1', 'CAST', '10.1016/j.msea.2019.138039') 2 [25.

,IDENTIFIER: Reference ID,FORMULA,PROPERTY: Microstructure,PROPERTY: Processing method,PROPERTY: BCC/FCC/other,PROPERTY: grain size ($\mu$m),PROPERTY: Exp. Density (g/cm$^3$),PROPERTY: Computed Density (g/cm$^3$),PROPERTY: HV,PROPERTY: Type of test,...,PROPERTY: Elongation (%),PROPERTY: Elongation plastic (%),PROPERTY: Exp. Young modulus (GPa),PROPERTY: Computed Young modulus (GPa),PROPERTY: O content (wppm),PROPERTY: N content (wppm),PROPERTY: C content (wppm),REFERENCE: doi,REFERENCE: year,REFERENCE: title
0,27,Al0.25 Co1 Fe1 Ni1,FCC,CAST,FCC,NaN,NaN,8.1,138.0,C,...,NaN,NaN,NaN,196.0,NaN,NaN,NaN,10.1016/j.jmmm.2014.07.023,2014,Effects of Al and Si addition on the structure...
1,27,Al0.5 Co1 Fe1 Ni1,FCC+BCC,CAST,other,NaN,NaN,7.7,212.0,C,...,NaN,NaN,NaN,187.0,NaN,NaN,NaN,10.1016/j.jmmm.2014.07.023,2014,Effects of Al and Si addition on the structure...
2,27,Al0.75 Co1 Fe1 Ni1,FCC+BCC,CAST,other,NaN,NaN,7.4,385.0,C,...,NaN,NaN,NaN,179.0,NaN,NaN,NaN,10.1016/j.jmmm.2014.07.023,2014,Effects of Al and Si addition on the structure...
3,27,Al1 Co1 Fe1 Ni1,BCC,CAST,BCC,NaN,NaN,7.1,456.0,C,...,NaN,NaN,NaN,172.0,NaN,NaN,NaN,10.1016/j.jmmm.2014.07.023,2014,Effects of Al and Si addition on the structure...
4,27,Co1 Fe1 Ni1,FCC,CAST,FCC,NaN,NaN,8.6,125.0,C,...,NaN,NaN,NaN,207.0,NaN,NaN,NaN,10.1016/j.jmmm.2014.07.023,2014,Effects of Al and Si addition on the structure...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1649,133,Al1 Nb1 Ti1 Zr1,NaN,NaN,other,NaN,5.79,5.6,539.6,C,...,5.9,NaN,NaN,90.0,NaN,NaN,NaN,10.1080/02670836.2018.1446267,2018,Microstructure and mechanical properties of a ...
1650,133,Hf1 Nb1 Ti1 Zr1,NaN,NaN,other,NaN,NaN,8.2,NaN,C,...,16.5,NaN,NaN,92.0,NaN,NaN,NaN,10.1080/02670836.2018.1446267,2018,Microstructure and mechanical properties of a ...
1651,133,Al0.667 Nb1 Ta0.333 Ti1 Zr0.333,NaN,NaN,other,NaN,6.80,6.8,407.9,C,...,4.8,NaN,NaN,106.0,NaN,NaN,NaN,10.1080/02670836.2018.1446267,2018,Microstructure and mechanical properties of a ...
1652,133,Nb1 Ti1 V1 Zr1,NaN,NaN,other,NaN,6.52,6.4,335.0,C,...,50.0,NaN,NaN,104.0,NaN,NaN,NaN,10.1080/02670836.2018.1446267,2018,Microstructure and mechanical properties of a ...


In [22]:
# query crossref
ref_data = {row['IDENTIFIER: Reference ID']: works.doi(row['REFERENCE: doi']) for index, row in df.iterrows()}
json.dump(ref_data, open('reference_data.json', 'w'))

In [23]:
# add refs not in cr
ref_data[48] = {'title':['Solution strengthening of ductile refractory HfMoxNbTaTiZr high-entropy alloys'],
                 'URL':'https://doi.org/10.1016/j.matlet.2016.03.133', 
                 'issued':{'date-parts':[[2016]]}} 

In [24]:

df['REFERENCE: year'] = df['IDENTIFIER: Reference ID'].apply(lambda x: str(ref_data[x]['issued']['date-parts'][0][0]) if ref_data[x] != None else None)
df['REFERENCE: title'] = df['IDENTIFIER: Reference ID'].apply(lambda x: ref_data[x]['title'][0] if ref_data[x] != None else None)
df['REFERENCE: url'] = df['IDENTIFIER: Reference ID'].apply(lambda x: ref_data[x]['URL'] if ref_data[x] != None else None)
df

,REFERENCE: tag,REFERENCE: doi,FORMULA,PROPERTY: Type of phases,PROPERTY: Single/Multiphase,PROPERTY: synthesis method,PROPERTY: grain size ($\mu$m),PROPERTY: ROM Density (g/cm$^3$),PROPERTY: HV,PROPERTY: Type of test,...,PROPERTY: C content (wppm),REFERENCE: comment,"Internal Reference # (J4, J90)","J4, J90 Original DOI",PROPERTY: Exp. Density (g/cm$^3$),Unnamed: 22,IDENTIFIER: Reference ID,REFERENCE: year,REFERENCE: title,REFERENCE: url
0,4,10.1016/j.jmmm.2014.07.023,Al0.25CoFeNi,FCC,S,AC,NaN,7.9,138,C,...,NaN,NaN,NaN,NaN,NaN,NaN,27,2014,Effects of Al and Si addition on the structure...,http://dx.doi.org/10.1016/j.jmmm.2014.07.023
9,5,10.1016/j.actamat.2014.08.026,CoCrFeNi,FCC,S,CR,24,8.2,NaN,T,...,NaN,elongation is total,NaN,NaN,NaN,NaN,37,2014,Temperature dependence of the mechanical prope...,http://dx.doi.org/10.1016/j.actamat.2014.08.026
16,6,10.1016/j.intermet.2015.01.004,CoCrFeNb0.103Ni,FCC+Laves (C14),M,AC,150-200,8.2,NaN,T,...,NaN,all YS recorded as +1 MPa,NaN,NaN,NaN,NaN,46,2015,Effects of Nb additions on the microstructure ...,http://dx.doi.org/10.1016/j.intermet.2015.01.004
22,7,10.1016/j.jallcom.2009.08.090,Al0.25CoCrFeNi,FCC,S,A,NaN,7.7,110,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,54,2009,Microstructure and mechanical property of as-c...,http://dx.doi.org/10.1016/j.jallcom.2009.08.090
42,8,10.1007/s11837-013-0753-6,Al0.5CoCrFeMo0.5Ni,FCC+Sigma,M,AC,NaN,7.7,425,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,63,2013,Effect of Aluminum Content on Microstructure a...,http://dx.doi.org/10.1007/s11837-013-0753-6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
941,J131,10.3390/e21010015,HfMoNbTaTiZr,BCC,S,AC,NaN,NaN,NaN,C,...,NaN,NaN,NaN,NaN,NaN,NaN,129,2018,"Effects of Mo, Nb, Ta, Ti, and Zr on Mechanica...",http://dx.doi.org/10.3390/e21010015
967,J132,10.3390/coatings9010016,CoCrFeNi,FCC,S,VHP,0.0276,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,130,2018,Microstructure and Properties of CoCrFeNi(WC) ...,http://dx.doi.org/10.3390/coatings9010016
970,J133,10.5755/j01.ms.25.3.19452,Al0.4FeCrCo1.5NiTi0.3,FCC + BCC,M,SPS,NaN,NaN,515 $\pm$ 16,C,...,NaN,NaN,NaN,NaN,NaN,NaN,131,2019,Enhancement of Mechanical Properties and Corro...,http://dx.doi.org/10.5755/j01.ms.25.3.19452
971,J134,10.1007/s11661-018-4646-8,HfNbTaTiZr,BCC,S,AC,NaN,NaN,NaN,T,...,NaN,Hot Isostatically Pressed,NaN,NaN,NaN,NaN,132,2018,Effect of Cold Deformation and Annealing on th...,http://dx.doi.org/10.1007/s11661-018-4646-8


In [25]:
cols = ['IDENTIFIER: Reference ID', 'REFERENCE: title', 'REFERENCE: year', 'REFERENCE: doi', 'REFERENCE: url']
df = df[cols]

df.sort_values('IDENTIFIER: Reference ID').to_csv('references.csv', index=False)